In [0]:
import tensorflow as tf
import meteo_path as me

In [0]:
import altair as alt
import pdvega

# Real Solutions

$$x^2 - 1 + z(7-\alpha x)$$

In [0]:
mod = tm.Model({
    'constants': {
        'alpha': 5.0
    },
    'parameters': {
        'z': 1
    },
    'variables': {
        'x': 1
    },
    'equations': {
        'eq1': 'x**2-1+z*(7-alpha*x)'
    }
})

In [0]:
pars = {'z': 0.0}
var0 = {'x': 5.0}
with sess.as_default():
    var1 = mod.solve_system(pars, var0, output=True)
    ret = mod.eval_system(pars, var1)
    print(var1)
    print(ret)

In [0]:
par0 = {'z': 0.0}
par1 = {'z': 1.0}
var0 = {'x': 5.0}
with sess.as_default():
    (t_path, par_path, var_path) = mod.homotopy_bde(par0, par1, var0,
                                                    solve=True, output=True)

In [0]:
pd.DataFrame({
    'param': par_path[:,0],
    'eqvar': var_path[:,0]
}).vgplot.scatter('param', 'eqvar')

# Complex Solutions

Real (with ifuzz = $\varepsilon$)
$$x^2 - 1 + z(7-5x) = \varepsilon i$$

Complex
$$a^2 - 1 + z(7-5a) - b^2 = 0$$
$$b(2a-5z) = \varepsilon$$

In [0]:
modc = tm.Model({
    'parameters': {
        'z': 1,
        'eps': 1
    },
    'variables': {
        'a': 1,
        'b': 1
    },
    'equations': {
        'eqr': 'a**2-1+z*(7-5*a)-b**2',
        'eqc': 'b*(2*a-5*z)-eps'
    }
})

In [0]:
pars = {'z': 0.0, 'eps': 0.0}
var0 = {'a': 1.1, 'b': 1.1}
with sess.as_default():
    sess.run(tf.global_variables_initializer())
    var1 = modc.solve_system(pars, var0, output=True)
    ret = modc.eval_system(pars, var1)

In [0]:
par0p = {'z': 0.0, 'eps': 0.001}
par1p = {'z': 1.0, 'eps': 0.001}
var0p = {'a': 1.0, 'b': 0.001}
par0n = {'z': 0.0, 'eps': 0.001}
par1n = {'z': 1.0, 'eps': 0.001}
var0n = {'a': -1.0, 'b': 0.001}
with sess.as_default():
    (t_path1, par_path1, var_path1) = modc.homotopy_bde(par0p, par1p, var0p)
    (t_path2, par_path2, var_path2) = modc.homotopy_bde(par0n, par1n, var0n)

In [0]:
(fig, (ax1, ax2)) = plt.subplots(ncols=2, figsize=(14, 6))
ax1.plot(var_path1[2::2,0], par_path1[2::2,0], c=cmap[0]);
ax1.plot(var_path2[2::2,0], par_path2[2::2,0], c=cmap[1]);
ax2.plot(var_path1[2::2,1], par_path1[2::2,0], c=cmap[0]);
ax2.plot(var_path2[2::2,1], par_path2[2::2,0], c=cmap[1]);

In [0]:
df1 = pd.DataFrame({
    'par': par_path1[2::2,0],
    'var1': var_path1[2::2,0],
    'var2': var_path1[2::2,1]
})
df2 = pd.DataFrame({
    'par': par_path2[2::2,0],
    'var1': var_path2[2::2,0],
    'var2': var_path2[2::2,1]
})

In [0]:
alt.vconcat(
    df1.vgplot('par', 'var1') + df2.vgplot('par', 'var1'),
    df1.vgplot('par', 'var2') + df2.vgplot('par', 'var2')
)

## Functional Approach

In [0]:
z = tf.Variable([1.0], name='z')
eps = tf.Variable([1.0], name='eps')

In [0]:
a = tf.Variable([1.0], name='a')
b = tf.Variable([1.0], name='b')

In [0]:
eqr = me.rename(a**2-1+z*(7-5*a)-b**2, 'real')
eqi = me.rename(b*(2*a-5*z)-eps, 'imag')

In [0]:
mod = me.Model([z, eps], [a, b], [eqr, eqi])

In [0]:
with tf.Session().as_default():
    tf.global_variables_initializer().run()
    mod.solve_system(output=True)
    ret = mod.eval_system()
me.summary(ret)